In [1]:
import rasterio


In [2]:
dataset = rasterio.open('../Files\LC08_L1TP_226087_20191005_20191018_01_T1_B4.TIF')

In [3]:
print('name: ',dataset.name)
print('mode: ',dataset.mode)
print('closed:',dataset.closed)

name:  ../Files\LC08_L1TP_226087_20191005_20191018_01_T1_B4.TIF
mode:  r
closed: False


Dataset attributes

In [4]:
print('count: ',dataset.count)
print('width: ',dataset.width)
print('height: ',dataset.height)

count:  1
width:  7891
height:  7921


A dataset band is an array of values representing the partial distribution of a single variable in 2-dimensional (2D) space. All band arrays of a dataset have the same number of rows and columns. The variable represented by the example dataset’s sole band is Level-1 digital numbers (DN) for the Landsat 8 band 4 (wavelengths between 640-670 nanometers). These values can be scaled to radiance or reflectance values. The array of DN values is 7897 columns wide and 7921 rows high.

In [5]:
dataset.bounds

BoundingBox(left=506685.0, bottom=-4426215.0, right=743415.0, top=-4188585.0)

In [6]:
dataset.transform

Affine(30.0, 0.0, 506685.0,
       0.0, -30.0, -4188585.0)

A dataset’s transform is an affine transformation matrix that maps pixel locations in (row, col) coordinates to (x, y) spatial positions. The product of this matrix and (0, 0), the row and column coordinates of the upper left corner of the dataset, is the spatial position of the upper left corner.

In [7]:
dataset.transform * (0, 0)

(506685.0, -4188585.0)

In [8]:
dataset.transform * (dataset.width, dataset.height)

(743415.0, -4426215.0)

But what do these numbers mean? 743415.0 meters from where? These coordinate values are relative to the origin of the dataset’s coordinate reference system (CRS).

In [9]:
dataset.crs

CRS.from_epsg(32620)

#Segun el ejemplo en los docs# “EPSG 32612” identifies a particular coordinate reference system: UTM zone 12N. This system is used for mapping areas in the Northern Hemisphere between 108 and 114 degrees west. The upper left corner of the example dataset, (358485.0, 4265115.0), is 141.5 kilometers west of zone 12’s central meridian (111 degrees west) and 4265 kilometers north of the equator.

Between the crs attribute and transform the georeferencing of a raster dataset is described and the dataset can compared to other GIS datasets.

# Reading raster data
Data from a raster band can be accessed by the band’s index number. Following the GDAL convention, bands are indexed from 1.

In [10]:
dataset.indexes

(1,)

In [11]:
band1 = dataset.read(1)

In [12]:
# The read() method returns a Numpy N-D array.
band1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [13]:
# Values from the array can be addressed by their row, column index.
band1[dataset.height // 2, dataset.width // 2]

9300

# Spatial indexing
Datasets have an index() method for getting the array indices corresponding to points in georeferenced space. To get the value for the pixel 100 kilometers east and 50 kilometers south of the dataset’s upper left corner, do the following.

In [14]:
x, y = (dataset.bounds.left + 100000, dataset.bounds.top - 50000)
row, col = dataset.index(x, y)
row, col

(1666, 3333)

In [15]:
band1[row, col]

9337

In [16]:
# To get the spatial coordinates of a pixel, use the dataset’s xy() method. The coordinates of the center of the image can be computed like this.
dataset.xy(dataset.height // 2, dataset.width // 2)

(625050.0, -4307400.0)

# Creating data
Reading data is only half the story. Using Rasterio dataset objects, arrays of values can be written to a raster data file and thus shared with other GIS applications such as QGIS.

As an example, consider an array of floating point values representing, e.g., a temperature or pressure anomaly field measured or modeled on a regular grid, 240 columns by 180 rows. The first and last grid points on the horizontal axis are located at 4.0 degrees west and 4.0 degrees east longitude, the first and last grid points on the vertical axis are located at 3 degrees south and 3 degrees north latitude.

In [17]:
import numpy as np
x = np.linspace(-4.0, 4.0, 240)
y = np.linspace(-3.0, 3.0, 180)
X, Y = np.meshgrid(x, y)
Z1 = np.exp(-2 * np.log(2) * ((X - 0.5) ** 2 + (Y - 0.5) ** 2) / 1 ** 2)
Z2 = np.exp(-3 * np.log(2) * ((X + 0.5) ** 2 + (Y + 0.5) ** 2) / 2.5 ** 2)
Z = 10.0 * (Z2 - Z1)

In [18]:
# In this example the coordinate reference system will be '+proj=latlong', which describes an equirectangular coordinate reference system with units of decimal degrees. The proper affine transformation matrix can be computed from the matrix product of a translation and a scaling.
from rasterio.transform import Affine
res = (x[-1] - x[0]) / 240.0
transform = Affine.translation(x[0] - res / 2, y[0] - res / 2) * Affine.scale(res, res)
transform

Affine(0.03333333333333333, 0.0, -4.016666666666667,
       0.0, 0.03333333333333333, -3.0166666666666666)

# Opening a dataset in writing mode
To save this array along with georeferencing information to a new raster data file, call rasterio.open() with a path to the new file to be created, 'w' to specify writing mode, and several keyword arguments.

- driver: the name of the desired format driver

- width: the number of columns of the dataset

- height: the number of rows of the dataset

- count: a count of the dataset bands

- dtype: the data type of the dataset

- crs: a coordinate reference system identifier or description

- transform: an affine transformation matrix, and

- nodata: a “nodata” value

The first 5 of these keyword arguments parametrize fixed, format-specific properties of the data file and are required when opening a file to write. The last 3 are optional.

A dataset for storing the example grid is opened like so:

In [19]:
new_dataset = rasterio.open(
    '../new.tif',
    'w',
    driver='GTiff',
    height=Z.shape[0],
    width=Z.shape[1],
    count=1,
    dtype=Z.dtype,
    crs='+proj=latlong',
    transform=transform,
    )

# Saving raster data
To copy the grid to the opened dataset, call the new dataset’s write() method with the grid and target band number as arguments.

In [20]:
new_dataset.write(Z, 1)

In [21]:
# Then call the close() method to sync data to disk and finish.
new_dataset.close()